In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd
import sys
import networkx as nx
import os
import warnings
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Experiment Graph
from execution_environment import ExecutionEnvironment as ee

# Suppress warnings
warnings.filterwarnings('ignore')
% matplotlib inline


try:
    import pygraphviz
    from networkx.drawing.nx_agraph import graphviz_layout
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import graphviz_layout
    except ImportError:
        raise ImportError("This example needs Graphviz and either "
                          "PyGraphviz or pydot")
        
def plot_graph():
    plt.figure(figsize=(12, 12))
    #pos = nx.spring_layout(ee.graph.graph)
    pos = graphviz_layout(ee.graph.graph, prog='twopi', args='')
    color_map = []
    for node in ee.graph.graph.nodes(data=True):
        if node[1]['root']:
            color_map.append('green')
        elif node[1]['type'] == 'Dataset' or node[1]['type'] == 'Feature':
            color_map.append('red')
        elif node[1]['type'] == 'Agg' or node[1]['type'] == 'SK_Model':
            color_map.append('blue')
        elif node[1]['type'] == 'SuperNode':
            color_map.append('grey')
        else:
            color_map.append('black')

    nx.draw(ee.graph.graph,
            node_color=color_map,
            pos=pos,
            node_size=100)
    nx.draw_networkx_edge_labels(ee.graph.graph,
                                 pos=pos,
                                 edge_labels={(u, v): d["name"] for u, v, d in ee.graph.graph.edges(data=True)})
    plt.show()

In [2]:
ROOT_DIRECTORY = '../data/home-credit-default-risk'
print(os.listdir(ROOT_DIRECTORY))

['application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [3]:
app_train = ee.load(ROOT_DIRECTORY + '/application_train.csv')
#print('Training data shape: ', app_train.shape().get())
#app_train.head().get()

In [4]:
app_test = ee.load(ROOT_DIRECTORY + '/application_test.csv')

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, Imputer

# Make the model with the specified regularization parameter
sk_imputer = Imputer(strategy = 'median')
sk_log_reg = LogisticRegression(C = 0.0001)
train = app_train[['CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_ANNUITY','FLAG_DOCUMENT_18']]
test = app_test[['CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_ANNUITY','FLAG_DOCUMENT_18']]
# Fit on the training data
imputer = train.fit_sk_model(sk_imputer)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(test)

train_labels = app_train['TARGET']
# Train on the training data
log_reg = train.fit_sk_model_with_labels(sk_log_reg, train_labels)

In [16]:
pred = log_reg.predict_proba(test)

In [27]:
pred.get()[:]

array([[ 0.87271708,  0.12728292],
       [ 0.81866031,  0.18133969],
       [ 0.98911762,  0.01088238],
       ..., 
       [ 0.9519514 ,  0.0480486 ],
       [ 0.94416837,  0.05583163],
       [ 0.89071093,  0.10928907]])

In [29]:
app_train.get()[1]

KeyError: 1